# Chapter 1. The World's Smallest Django Project

---

# TOC

**1. Hello Django**
    1. Creating the View
    2. The URL Patterns
    3. The Settings
    4. Running the Example
**2. Improvements**
    1. WSGI Application
    2. Additional Configuration
    3. Reusable Template

---

## 1. Hello Django

- While the **startproject** command is a useful tool, it is not required in order to start a Django project.
- This lightweight “Hello World” project will create a simple Django application **using a single-file approach.**
- In this chapter, we’ll start by using a single ***hello.py*** file. This file will contain all of the code needed to run our Django project.

- `django-admin.py startproject 프로젝트이름`
    - 필요한 프로젝트 디렉토리와 파일 자동 생성
- 하지만 개발자들의 편의를 위해 파일을 나누고 구조를 잡는 것이지 하나의 파일에서 구현가능함

### 1) Creating the View

![](http://blog.easylearning.guru/wordpress/wp-content/uploads/2015/08/Django-Template.png)

model-template-view (MTV) framework
- The view portion typically
    - **inspects the incoming HTTP request**, 
    - **queries the necessary data** to send to the presentation layer.
    - constructs the necessary data to send to the presentation layer.

In [1]:
%%writefile hello.py
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

Writing hello.py


- This would typically be in a **`views.py`** file inside one of your apps.
- There is also no requirement that views live in a file called views.py.

### 2) The URL Patterns

- Django associates views with their URL by pairing a regular expression to match the URL and any callable arguments to the view.
- The following is an example from hello.py of how we make this connection.

In [2]:
%%writefile hello.py
from django.conf.urls import url
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

Overwriting hello.py


- Now this file combines both a typical views.py file and the root urls.py file.
- Again, it is worth noting that there is **no requirement for the URL patterns to be included in a urls.py file.**

### 3) The Settings

- **Django settings** detail everything from database and cache connections to internationalization features and static and uploaded resources.
- For many developers just getting started, **the settings in Django are a major point of confusion.**
- While **recent releases have worked to trim down the default settings’ file length**, it can still be overwhelming.

In [3]:
%%writefile hello.py
from django.conf.urls import url
from django.http import HttpResponse
from django.conf import settings

settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

Overwriting hello.py


- A **secret key** must be generated for the default session and cross-site request forgery (CSRF) protection.
- It is important for any production site to have a **random SECRET_KEY** that is kept private.
- https://docs.djangoproject.com/en/1.7/topics/signing

### 4) Running the Example

In [4]:
%%writefile hello.py
import sys
from django.http import HttpResponse
from django.conf import settings
from django.conf.urls import url


# settings
settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
def index(request):
    return HttpResponse('Hello World')

# urls
urlpatterns = (
    url(r'^$', index),
)

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Overwriting hello.py


`$ python hello.py runserver`

---

## 2. Improvements

Django also provides **ADDITIONAL UTILITIES** for common tasks such as:
- rendering HTML
- parsing form data
- persisting session state

### 1) WSGI Application

#### runserver

- It has helpful utilities for **local development** such as auto–code reloading.
- While it is convenient for local development, runserver is **not appropriate for production deployment** security.

#### Web Server Gateway Interface(WSGI)

is **the specification for how web servers communicate with application frameworks such as Django**, and was defined by PEP 333 and improved in PEP 3333.

- mod_wsgi
- Gunicorn
- uWSGI
- CherryPy
- Tornado
- Chaussette

![](http://www.nowamagic.net/librarys/images/201309/2013_09_04_01.png)

![](http://brianmcdonnell.github.io/pycon_ie_2013/images/diagrams/min_wsgi_stack.png)

- Each of these servers needs a **properly defined WSGI application** to be used. Django has an easy interface for creating this application through **get_wsgi_application**.

In [5]:
%%writefile hello.py
import sys
from django.conf.urls import url
from django.http import HttpResponse
from django.conf import settings
from django.core.wsgi import get_wsgi_application


# settings
settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
def index(request):
    return HttpResponse('Hello World')

# urls
urlpatterns = (
    url(r'^$', index),
)

# wsgi
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Overwriting hello.py


`application = get_wsgi_application()`의 경우 settings.configure 이 후 시점에 해야 작동된다.

This would normally be contained within the **wsgi.py** file created by the startproject command.

##### Gunicorn

**Gunicorn** is a **popular choice** for a pure-Python WSGI application server. (2015.09.02 버전 19.3.0)

```bash
$ pip install gunicorn
```

You can **run** it fairly simply

```bash
$ gunicorn hello --log-file=-
```

[docs.gunicorn.org](http://docs.gunicorn.org)

---

### 2) Additional Configuration

- The Twelve Factor App 에서 development, production, staging 단계에서의 프로젝트 세팅에 대한 방법론을 제시하고 있다.
- 이곳에서 확인가능 -  http://12factor.net/ko/dev-prod-parity
- configuration을 분리하고 환경 변수를 저장하도록 권장

**While Gunicorn is a production-ready web server, the application itself is not yet production ready**, as **DEBUG** should never be enabled in production.

1. DEBUG
2. SECRET_KEY
3. ALLOWED_HOSTS

In [6]:
%%writefile hello.py
import os
import sys
from django.conf import settings
from django.http import HttpResponse
from django.conf.urls import url
from django.core.wsgi import get_wsgi_application


# settings
DEBUG = os.environ.get('DEBUG', 'on') == 'on'
SECRET_KEY = os.environ.get('SECRET_KEY', os.urandom(32))

settings.configure(
    DEBUG=DEBUG,
    SECRET_KEY=SECRET_KEY,
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
def index(request):
    return HttpResponse('He
                        llo World')

# urls
urlpatterns = (
    url(r'^$', index),
)

# wsgi
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Overwriting hello.py


>[Official doc.: os module](https://docs.python.org/3/library/os.html)

>- **os.environ**: A mapping object representing the string environment. For example, environ['HOME'] is the pathname of your home directory (on some platforms), and is equivalent to getenv("HOME") in C.

>- **os.urandom(n)**: Return a string of n random bytes suitable for cryptographic use.

The default for **DEBUG** is True, and the **SECRET_KEY** will be randomly generated each time the application is loaded if it is not set.

```bash
$ export DEBUG=off
$ python hello.py runserver
```

```bash
CommandError: You must set settings.ALLOWED_HOSTS if DEBUG is False.
```

- Production service에서는 **ALLOWDED_HOSTS** 설정하기

In [7]:
%%writefile hello.py
import os
import sys
from django.conf import settings
from django.http import HttpResponse
from django.conf.urls import url
from django.core.wsgi import get_wsgi_application


# settings
DEBUG = os.environ.get('DEBUG', 'on') == 'on'
SECRET_KEY = os.environ.get('SECRET_KEY', os.urandom(32))
ALLOWED_HOSTS = os.environ.get('ALLOWED_HOSTS', 'localhost').split(',')

settings.configure(
    DEBUG=DEBUG,
    SECRET_KEY=SECRET_KEY,
    ALLOWED_HOSTS=ALLOWED_HOSTS,
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
def index(request):
    return HttpResponse('Hello World')

# urls
urlpatterns = (
    url(r'^$', index),
)

# wsgi
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Overwriting hello.py


### settings.py 에서 환경 변수 설정하는 다른 방법

virtualenvwrapper를 사용하고 있다면

1. postactivate
2. predeactivate

를 이용해서 필요한 환경 변수를 미리 설정하여 사용한다. 예를 들어 DEBUG, SECRET_KEY, ALLOWED_HOSTS, DJANGO_SETTINGS_MODULE, DATABASE_NAME, DATABASE_USER, DATABASE_PASSWORD

In [ ]:
from django.core.exceptions import ImproperlyConfigured

def get_env_variable(var_name, default):
    try:
        return os.environ.get(var_name, default)
    except KeyError:
        error_msg = "Set the %s environment variable" % var_name
        raise ImproperlyConfigured(error_msg)

DEBUG = get_env_variable('DEBUG', 'on') == 'on'
SECRET_KEY = get_env_variable('SECRET_KEY', os.urandom(32))
ALLOWED_HOSTS = get_env_variable('ALLOWED_HOSTS', 'localhost').split(',')

### 3) Reusable Template

So far this example has centered on **rethinking the layout created by Django’s startproject command**.

**A template for startproject is a directory or zip file** that is rendered as a Django template when the command is run.

To transform hello.py into **a project template (project_name/ project_name.py)**, the relevant parts of the file need to be replaced by these variables.

In [8]:
!mkdir -p project_name

In [9]:
!ls

__pycache__          hello.py             part2_chapter1.ipynb project_name


In [10]:
%%writefile project_name/project_name.py
import os
import sys
from django.conf import settings
from django.http import HttpResponse
from django.conf.urls import url
from django.core.wsgi import get_wsgi_application


# settings
DEBUG = os.environ.get('DEBUG', 'on') == 'on'
SECRET_KEY = os.environ.get('SECRET_KEY', os.urandom(32))
ALLOWED_HOSTS = os.environ.get('ALLOWED_HOSTS', 'localhost').split(',')

settings.configure(
    DEBUG=DEBUG,
    SECRET_KEY=SECRET_KEY,
    ALLOWED_HOSTS=ALLOWED_HOSTS,
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
def index(request):
    return HttpResponse('Hello World')

# urls
urlpatterns = (
    url(r'^$', index),
)

# wsgi
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Writing project_name/project_name.py


Rather than using os.urandom for the SECRET_KEY default, **this code will generate a random secret to be the default each time a new project is created.** This makes the SECRET_KEY default stable at the project level while still being sufficiently random across projects.

In [11]:
!django-admin.py startproject foo --template=project_name

In [13]:
!tree

.
├── __pycache__
│   └── hello.cpython-35.pyc
├── foo
│   └── foo.py
├── hello.py
├── part2_chapter1.ipynb
└── project_name
    └── project_name.py

3 directories, 5 files
